In [35]:
class MessageData():

    def __init__(self) -> None:
        self.ch = ''

        self.typ = ''

        self.no_note = ''
        self.no_velocity = ''
        self.no_time = ''

        self.cc_control = ''
        self.cc_value = ''
        self.cc_time = ''

        self.pw_pitch = ''
        self.pw_time = ''

        self.program_change = ''

        self.meta = ''


In [36]:
from mido import MidiFile, MidiTrack, Message

def preprocess(filename =  'HotelCalifornia.mid'):
    data = []
    mid = MidiFile(filename)
    speed = mid.ticks_per_beat  


    for n, track in enumerate(mid.tracks[1:]):
        typ = ''

        no_note = ''
        no_velocity = ''
        no_time = ''
        lno = 0

        cc_control = ''
        cc_value = ''
        cc_time = ''
        lcc = 0

        pw_pitch = ''
        pw_time = ''
        lpw = 0

        program_change = ''

        for i in track:
            if i.type == "note_on":
                lno += 1
                typ += 'no'+' '
                no_note += str(i.note)+' '
                no_velocity += str(i.velocity)+' '
                no_time += str(i.time)+ ' '
            elif i.type == "control_change":
                lcc += 1
                typ += 'cc'+' '
                cc_control += str(i.control)+' '
                cc_value += str(i.value)+' '
                cc_time += str(i.time)+' '
                #print(i)
            elif i.type == "program_change":
                program_change += str(i.program)+' '+str(i.time)+"\n"
                #print('i')
            elif i.type == "pitchwheel":
                lpw += 1
                typ += 'pw'+' '
                pw_pitch += str(i.pitch)+' '
                pw_time += str(i.time)+' '
            elif i.is_meta:
                pass
                #print(i)
            else:
                print(i, "at track", n)

        for i in mid.tracks[0]:
            if i.type == "set_tempo":
                tempo = i.tempo
            elif i.type == "time_signature":
                nom = i.numerator
                denom = i.denominator
            elif i.type == "key_signature":
                key = i.key

        msg = MessageData()
        msg.ch = n
        msg.typ = typ
        msg.no_note = no_note
        msg.no_velocity = no_velocity
        msg.no_time = no_time
        msg.cc_control = cc_control
        msg.cc_value = cc_value
        msg.cc_time = cc_time
        msg.pw_pitch = pw_pitch
        msg.pw_time = pw_time
        msg.meta = f"{lno+lcc} {lno} {lcc}\n" + program_change

        data.append(msg)

        
    return data, len(mid.tracks)-1, speed, tempo, nom, denom, key



In [37]:
import re
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
token_type = 'word'
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, LSTM, Input, Embedding, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import LambdaCallback

def lstm(txt, lenout):
    text = txt

    seq_length = 100
    start_story = '| ' * seq_length
    text = start_story + text
    text = text.lower()
    text = text.replace('\n\n\n\n\n', start_story)
    text = text.replace('\n', ' ')
    text = re.sub('  +', '. ', text).strip()
    text = text.replace('..', '.')
    text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
    text = re.sub('\s{2,}', ' ', text)

    if token_type == 'word':
        tokenizer = Tokenizer(char_level = False, filters = '')
    else:
        tokenizer = Tokenizer(char_level = True, filters = '', lower = False)
    tokenizer.fit_on_texts([text])
    total_words = len(tokenizer.word_index) + 1
    token_list = tokenizer.texts_to_sequences([text])[0]


    def generate_sequences(token_list, step): 
        X = []
        y = []
        for i in range(0, len(token_list) - seq_length, step):
            X.append(token_list[i: i + seq_length])
            y.append(token_list[i + seq_length])
        y = to_categorical(y, num_classes = total_words)
        num_seq = len(X)
        print('Number of sequences:', num_seq, "\n")
        return X, y, num_seq

    step = 1
    seq_length = seq_length

    X, y, num_seq = generate_sequences(token_list, step)

    X = np.array(X)
    y = np.array(y)

    n_units = 256
    embedding_size = 100

    text_in = Input(shape = (None,))
    embedding = Embedding(total_words, embedding_size)
    x = embedding(text_in)
    x = LSTM(n_units)(x)
    # x = Dropout(0.2)(x)
    text_out = Dense(total_words, activation = 'softmax')(x)

    model = Model(text_in, text_out)

    opti = RMSprop(lr = 0.005)
    model.compile(loss='categorical_crossentropy', optimizer=opti)

    #model.summary()

    def sample_with_temp(preds, temperature=1.0):
        # helper function to sample an index from a probability array
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)

    def generate_text(seed_text, next_words, model, max_sequence_len, temp):
        output_text = seed_text
        
        seed_text = start_story + seed_text
        
        for _ in range(next_words):
            token_list = tokenizer.texts_to_sequences([seed_text])[0]
            token_list = token_list[-max_sequence_len:]
            token_list = np.reshape(token_list, (1, max_sequence_len))
            
            probs = model.predict(token_list, verbose=0)[0]
            y_class = sample_with_temp(probs, temperature = temp)
            
            if y_class == 0:
                output_word = ''
            else:
                output_word = tokenizer.index_word[y_class]
                
            if output_word == "|":
                break
                
            if token_type == 'word':
                output_text += output_word + ' '
                seed_text += output_word + ' '
            else:
                output_text += output_word + ' '
                seed_text += output_word + ' '             
        return output_text

    def on_epoch_end(epoch, logs):
        seed_text = ""
        gen_words = 300

        #print('Temp 0.2')
        generate_text(seed_text, gen_words, model, seq_length, temp = 0.2)
        #print('Temp 0.33')
        generate_text(seed_text, gen_words, model, seq_length, temp = 0.33)
        #print('Temp 0.5')
        generate_text(seed_text, gen_words, model, seq_length, temp = 0.5)
        #print('Temp 1.0')
        generate_text(seed_text, gen_words, model, seq_length, temp = 1)

    epochs = 10
    batch_size = 32
    num_batches = int(len(X) / batch_size)
    callback = LambdaCallback(on_epoch_end=on_epoch_end)
    model.fit(X, y, epochs=epochs, batch_size=batch_size, callbacks = [callback], shuffle = True)

    def generate_human_led_text(model, max_sequence_len):
    
        output_text = ''
        seed_text = start_story
        
        from random import randint, random
        from tqdm.notebook import tqdm
        for _ in tqdm(range(int(lenout))):
            token_list = tokenizer.texts_to_sequences([seed_text])[0]
            token_list = token_list[-max_sequence_len:]
            token_list = np.reshape(token_list, (1, max_sequence_len))
            
            probs = model.predict(token_list, verbose=0)[0]

            top_10_idx = np.flip(np.argsort(probs)[-10:])
            top_10_probs = [probs[x] for x in top_10_idx]
            top_10_words = tokenizer.sequences_to_texts([[x] for x in top_10_idx])

            r = random()
            if  1 >= r >= 1 - top_10_probs[0]:
                index = 0
            elif 1 - top_10_probs[0] > r >= 1 - top_10_probs[0] - top_10_probs[1]:
                index = 1
            elif 1 - top_10_probs[0] - top_10_probs[1] > r > 1 - top_10_probs[0] - top_10_probs[1] - top_10_probs[2]:
                index = 2
            else:
                index = 3
                #print(f"outlier!: {index}, {r}")

            chosen_word = top_10_words[index]
                
            
            seed_text += chosen_word + ' '
            output_text += chosen_word + ' '

        #print (output_text)
        return output_text

    t = generate_human_led_text(model, seq_length)

    return t


In [63]:
from mido import MidiFile, MidiTrack, Message, MetaMessage

def channel_assemble(data, ch):
    trackdata = data[ch]

    typ = trackdata.typ.split()
    meta = trackdata.meta.split('\n')
    program = meta[1].split() 
    cc_c = trackdata.cc_control.split()
    cc_t = trackdata.cc_time.split()
    cc_v = trackdata.cc_value.split()
    no_n = trackdata.no_note.split()
    no_t = trackdata.no_time.split()
    no_v = trackdata.no_velocity.split()
    pw_p = trackdata.pw_pitch.split()
    pw_t = trackdata.pw_time.split()

    noindx, ccindx, pwindx = 0, 0, 0
    track = MidiTrack()
    if program: 
        m = Message("program_change", channel = ch, program=int(program[0]), time=int(program[1]))
        track.append(m)
    for node in typ:
        if node == "no":
            m = Message("note_on", channel = ch, note=int(no_n[noindx]), time=int(no_t[noindx]), velocity=int(no_v[noindx]))
            noindx += 1
        elif node == "cc":
            m = Message("control_change", channel = ch, control=int(cc_c[ccindx]), time=int(cc_t[ccindx]), value=int(cc_v[ccindx]))
            ccindx += 1
        elif node == "pw":
            m=Message("pitchwheel", channel = ch, pitch=int(pw_p[pwindx]), time=int(pw_t[pwindx]))
            pwindx += 1
        else:
            print("Unknown node", node)
        track.append(m)
        
    return track

def postprocessing(data, lentrack, speed=120, tempo = 810810, nom = 4, denom = 4, k = 'C', select = []):

    newmid = MidiFile()
    newmid.ticks_per_beat=speed
    metatrack = MidiTrack()
    metatrack.append(MetaMessage("set_tempo", tempo=int(tempo)))
    metatrack.append(MetaMessage("time_signature", numerator = int(nom), denominator = int(denom)))
    metatrack.append(MetaMessage("key_signature", key=k))
    newmid.tracks.append(metatrack)

    if select:
        for i in range(select):
            newmid.tracks.append(channel_assemble(data, i))
        newmid.save("output.mid")
        return
        
    for i in range(lentrack):
        newmid.tracks.append(channel_assemble(data, i))

    newmid.save("output.mid")






In [39]:
def train_note_channel(data, ch):
    notelen = len(data[ch].no_note)
    data[ch].no_note =  lstm(data[ch].no_note, int(notelen))

In [40]:
filename = 'HotelCalifornia.mid'

In [41]:
data, n, speed, tempo, nom, denom, key= preprocess(filename)

In [42]:
for i in (range(len(data))):
    train_note_channel(data, i)

Number of sequences: 9908 

Epoch 1/10
310/310 [==============================] - 162s 521ms/step - loss: 1.5686
Epoch 2/10
310/310 [==============================] - 235s 760ms/step - loss: 0.5379
Epoch 3/10
310/310 [==============================] - 253s 816ms/step - loss: 0.3156
Epoch 4/10
310/310 [==============================] - 228s 736ms/step - loss: 0.2249
Epoch 5/10
310/310 [==============================] - 232s 750ms/step - loss: 0.1713
Epoch 6/10
310/310 [==============================] - 235s 759ms/step - loss: 0.1425
Epoch 7/10
310/310 [==============================] - 233s 753ms/step - loss: 0.1150
Epoch 8/10
310/310 [==============================] - 240s 776ms/step - loss: 0.1023
Epoch 9/10
310/310 [==============================] - 240s 775ms/step - loss: 0.0938
Epoch 10/10
310/310 [==============================] - 242s 782ms/step - loss: 0.0871


  0%|          | 0/29724 [00:00<?, ?it/s]

Number of sequences: 1044 

Epoch 1/10
33/33 [==============================] - 57s 2s/step - loss: 2.1756
Epoch 2/10
33/33 [==============================] - 70s 2s/step - loss: 1.1593
Epoch 3/10
33/33 [==============================] - 72s 2s/step - loss: 0.7107
Epoch 4/10
33/33 [==============================] - 69s 2s/step - loss: 0.4440
Epoch 5/10
33/33 [==============================] - 69s 2s/step - loss: 0.2548
Epoch 6/10
33/33 [==============================] - 69s 2s/step - loss: 0.1917
Epoch 7/10
33/33 [==============================] - 69s 2s/step - loss: 0.1407
Epoch 8/10
33/33 [==============================] - 67s 2s/step - loss: 0.1416
Epoch 9/10
33/33 [==============================] - 69s 2s/step - loss: 0.0880
Epoch 10/10
33/33 [==============================] - 68s 2s/step - loss: 0.0572


  0%|          | 0/3132 [00:00<?, ?it/s]

Number of sequences: 1592 

Epoch 1/10
50/50 [==============================] - 78s 2s/step - loss: 2.0791
Epoch 2/10
50/50 [==============================] - 73s 1s/step - loss: 1.1189
Epoch 3/10
50/50 [==============================] - 66s 1s/step - loss: 0.6759
Epoch 4/10
50/50 [==============================] - 67s 1s/step - loss: 0.4252
Epoch 5/10
50/50 [==============================] - 67s 1s/step - loss: 0.2658
Epoch 6/10
50/50 [==============================] - 68s 1s/step - loss: 0.1971
Epoch 7/10
50/50 [==============================] - 66s 1s/step - loss: 0.1590
Epoch 8/10
50/50 [==============================] - 67s 1s/step - loss: 0.0960
Epoch 9/10
50/50 [==============================] - 67s 1s/step - loss: 0.0943
Epoch 10/10
50/50 [==============================] - 66s 1s/step - loss: 0.0959


...

  0%|          | 0/4890 [00:00<?, ?it/s]

Number of sequences: 2516 

Epoch 1/10
79/79 [==============================] - 99s 1s/step - loss: 0.9499
Epoch 2/10
79/79 [==============================] - 88s 1s/step - loss: 0.6357
Epoch 3/10
79/79 [==============================] - 86s 1s/step - loss: 0.5454
Epoch 4/10
79/79 [==============================] - 88s 1s/step - loss: 0.4759
Epoch 5/10
79/79 [==============================] - 90s 1s/step - loss: 0.4231
Epoch 6/10
79/79 [==============================] - 88s 1s/step - loss: 0.3486
Epoch 7/10
79/79 [==============================] - 86s 1s/step - loss: 0.2914
Epoch 8/10
79/79 [==============================] - 89s 1s/step - loss: 0.2381
Epoch 9/10
79/79 [==============================] - 87s 1s/step - loss: 0.2228
Epoch 10/10
79/79 [==============================] - 97s 1s/step - loss: 0.1923


  0%|          | 0/7548 [00:00<?, ?it/s]

Number of sequences: 0 

Epoch 1/10


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [46]:
len(data)

7

In [43]:
postprocessing(data, n, speed, tempo, nom, denom, key)

In [48]:
rawdata, n, speed, tempo, nom, denom, key = preprocess()

In [50]:
from copy import deepcopy

In [103]:
copydata = deepcopy(data)
maintain = 6
copydata[maintain] = rawdata[maintain]
#postprocessing(copydata, len(copydata), speed, tempo, nom, denom, key)

In [104]:
#copydata = deepcopy(data)
#maintain = 4
#copydata[maintain] = rawdata[maintain]
postprocessing(copydata, len(copydata), speed, tempo, nom, denom, key)

In [105]:
postprocessing(copydata, 5, speed, tempo, nom, denom, key)

In [76]:
copydata[4].meta = rawdata[4].meta

'1663 1630 33\n30 0\n'

In [75]:
copydata[5].meta

'2553 2516 37\n'